# Лабораторная работа №3
### Выполнил студент группы БВТ1905 Нестеров Юрий
## Задание №1

In [2]:
from termcolor import colored
import ipywidgets as wg
import random
import time


user_string = wg.Textarea(value='', placeholder='', description='Введите строку: ', disabled=False)  # User inputs in notebook
user_substring = wg.Text(value='', placeholder='', description='Введите подстроку: ', disabled=False)
user_letter_case = wg.Checkbox(value=False, description="Учитывать регистр", disabled=False, indent=False)

### Алгоритм поиска Кнута-Морриса-Пратта

In [5]:
def reversed(str):
    return str[::-1]


def pratt(sub, string):
    while True:
        symbol = chr(random.randint(0, 512))
        if symbol not in sub + string:
            break
    full = sub + symbol + string
    iter_array = [0] * len(full)
    for i in range(1, len(full)):
        search, reverse = full[0:i], full[i::-1]
        for j in range(1, len(search)):
            if search[0:j] == reversed(reverse[0:j]):
                iter_array[i] = j
    for i in range(len(iter_array)):
        if iter_array[i] == len(sub):
            print("Нашёл совпадение на " + str(i) + "-том шагу")
            print(full[i-len(sub)+1:i+1])


display(user_substring, user_string)
pratt(user_substring.value, user_string.value)

Text(value='ipsum', description='Введите подстроку: ', placeholder='')

Textarea(value='Lorem ipsum bla ipsum', description='Введите строку: ', placeholder='')

Нашёл совпадение на 16-том шагу
ipsum
Нашёл совпадение на 26-том шагу
ipsum


### Упрощённый алгоритм Бойера-Мура

In [14]:
def mour(sub, string):
    alphabet_dict = {}
    count = 0; res = []
    for i in sub[::-1]:
        if i not in alphabet_dict:
            alphabet_dict[i] = count
        count += 1
    match = len(sub)-1
    while match < len(string):
        if sub[-1] == string[match]:
            c = True
            for i in range(1, len(sub)):
                if sub[-1-i] != string[match-i]:
                    c = False
                    break
            if c:
                res.append(match-len(sub))
            match += 1
        elif alphabet_dict.get(string[match]) is not None:
            match += alphabet_dict.get(string[match])
        else:
            match += len(sub)
    return res
        

display(user_substring, user_string)
if len(user_substring.value) + len(user_string.value) > 0:
    print(mour(user_substring.value, user_string.value))

Text(value='Hooligan', description='Введите подстроку: ', placeholder='')

Textarea(value='Hoola-Hoola girls like Hooligans Hooligan', description='Введите строку: ', placeholder='')

[22, 32]
